In [1]:
import numpy as np
import os
import re
import pandas as pd
from collections import defaultdict
import matplotlib.pyplot as plt
import random
from scipy.spatial import KDTree

import my_utils as myu

import torch
import torch.nn.functional as F
import torch_geometric.transforms as T
# from torch_geometric.nn import SplineConv
from torch_geometric.typing import WITH_TORCH_SPLINE_CONV
from torch_geometric.data import Data

/jet/home/ussqww/.conda/envs/pyg/lib/python3.12/site-packages/torch_geometric/typing.py:54: UserWarning: An issue occurred while importing 'pyg-lib'. Disabling its usage. Stacktrace: /lib64/libm.so.6: version `GLIBC_2.29' not found (required by /ocean/projects/eng170006p/ussqww/.conda/envs/pyg/lib/python3.12/site-packages/libpyg.so)
  warnings.warn(f"An issue occurred while importing 'pyg-lib'. "
/jet/home/ussqww/.conda/envs/pyg/lib/python3.12/site-packages/torch_geometric/typing.py:110: UserWarning: An issue occurred while importing 'torch-sparse'. Disabling its usage. Stacktrace: /lib64/libm.so.6: version `GLIBC_2.29' not found (required by /ocean/projects/eng170006p/ussqww/.conda/envs/pyg/lib/python3.12/site-packages/libpyg.so)
  warnings.warn(f"An issue occurred while importing 'torch-sparse'. "


In [2]:
# check for valid spline_conv
if not WITH_TORCH_SPLINE_CONV:
    quit("This example requires 'torch-spline-conv'")

# Check available GPUs and prepare device list
if torch.cuda.is_available():
    devices = [f'cuda:{i}' for i in range(torch.cuda.device_count())]
else:
    devices = ['cpu']  # fallback to CPU if no GPUs are available
print("Using devices:", devices)

Using devices: ['cuda:0']


In [3]:
def sort_key_func(filename):
    """
    Extracts numbers from a filename and converts them to an integer for sorting.
    """
    numbers = re.findall(r'\d+', filename)
    return int(numbers[0]) if numbers else float('inf')

def read_vtk_file_pair(folder_path):
    """
    Reads the first and last VTK files in the specified folder that start with "physics_allparticles"
    and end with ".vtk", sorted by numeric order in filenames.

    Args:
        folder_path (str): Path to the folder containing VTK files.

    Returns:
        tuple: A tuple containing data from the middle and last VTK files.
    """
    # Filter files that start with "physics_allparticles" and end with ".vtk"
    vtk_files = [file for file in os.listdir(folder_path) 
                 if file.startswith("physics_allparticle") and file.endswith(".vtk")]
    
    vtk_files_sorted = sorted(vtk_files, key=sort_key_func)

    if not vtk_files_sorted:
        return None

    # Calculate the middle index
    middle_index = len(vtk_files_sorted) // 2

    # Read the middle and the last files
    middle_file_path = os.path.join(folder_path, vtk_files_sorted[middle_index])
    # last_file_path = os.path.join(folder_path, vtk_files_sorted[-1])
    last_file_path = os.path.join(folder_path, vtk_files_sorted[middle_index+2])

    # Assuming `read_mesh_allCon` is defined to read VTK files
    first_data = myu.read_mesh_allCon(middle_file_path, 3)
    last_data = myu.read_mesh_allCon(last_file_path, 3)

    return (first_data, last_data)


def read_all_folders_vtk_pairs(root_folder):
    """
    Reads the first and last VTK files from each subfolder within the root folder that start with "io2D".

    Args:
        root_folder (str): Path to the root folder containing subfolders with VTK files.

    Returns:
        list: A list of tuples, each containing data from the first and last VTK files from each subfolder.
    """
    folders = [f for f in os.listdir(root_folder) if f.startswith("io2D")]
    # folders = folders[:7] # for testing
    # print(folders)
    total_folders = len(folders)
    vtk_pairs = []
    
    for index, folder in enumerate(folders):
        outputs_path = os.path.join(root_folder, folder, "outputs")
        if os.path.isdir(outputs_path):
            vtk_pair = read_vtk_file_pair(outputs_path)
            if vtk_pair:
                vtk_pairs.append(vtk_pair)

        # Calculate and print the progress percentage
        progress_percent = ((index + 1) / total_folders) * 100
        print(f"Processing folder {index + 1}/{total_folders} ({progress_percent:.2f}%) completed: Reading from: {outputs_path}")

    return vtk_pairs

# root_folder = "/ocean/projects/eng170006p/ussqww/TTNG_05052024"  
# root_folder = "E:/NeuronGrowth_2024/TTNG_05052024"  
root_folder = "./dataset/NDDs" 
vtk_pairs = read_all_folders_vtk_pairs(root_folder)
for pair in vtk_pairs:
    first_data, last_data = pair
    # print("First File Data:", first_data)
    # print("Last File Data:", last_data)

Processing folder 1/4 (25.00%) completed: Reading from: ./dataset/NDDs/io2D_1c03/outputs
Processing folder 2/4 (50.00%) completed: Reading from: ./dataset/NDDs/io2D_1c02/outputs
Processing folder 3/4 (75.00%) completed: Reading from: ./dataset/NDDs/io2D_1c01/outputs
Processing folder 4/4 (100.00%) completed: Reading from: ./dataset/NDDs/io2D_2c01/outputs


In [4]:
def calculate_pseudo_coordinates(points, edge_index):
    """Calculate pseudo-coordinates for each edge based on node coordinates."""
    pseudo_coords = []
    for src, dest in edge_index.t().tolist():
        delta_x = points[dest, 0] - points[src, 0]
        delta_y = points[dest, 1] - points[src, 1]
        pseudo_coords.append([delta_x, delta_y])

    pseudo_coords = torch.tensor(pseudo_coords, dtype=torch.float)
    return pseudo_coords

def interpolate_features(current_points, current_point_data, next_points, k=3):
    """
    Interpolate feature data from current points to next points using k-nearest neighbors.

    Args:
        current_points (array): Coordinates of current points where data is known.
        current_point_data (dict): Dictionary mapping feature names to arrays of values.
        next_points (array): Coordinates of next points where data needs to be interpolated.
        k (int): Number of nearest neighbors to consider for interpolation.

    Returns:
        dict: A dictionary with interpolated features for each point in next_points.
    """
    # Create KDTree from current points
    tree = KDTree(current_points)
    interpolated_data = {key: np.zeros(len(next_points)) for key in current_point_data.keys()}

    # Iterate over each next point and interpolate features from nearest current points
    for i, point in enumerate(next_points):
        dists, indices = tree.query(point, k=k)  # Find k nearest neighbors
        
        # Handle cases where less than k points are available
        if not isinstance(indices, np.ndarray):
            indices = [indices]
            dists = [dists]

        weights = 1 / np.maximum(dists, 1e-6)  # Calculate weights inversely proportional to distance
        weight_sum = np.sum(weights)
        
        # Calculate weighted average for each feature
        for key in current_point_data.keys():
            # Extract the specific feature values for the nearest points
            feature_values = current_point_data[key][indices]
            # Compute the weighted average of the feature
            interpolated_data[key][i] = np.dot(weights, feature_values) / weight_sum

    return interpolated_data

def create_graph_data(points, point_data, edge_attributes, y_values):
    """
    Create graph data from provided points, elements, point data, edge attributes, and target labels.

    Args:
        points (array): Coordinates of points.
        point_data (dict): Features for each point, expected to be a dict of arrays.
        edge_attributes (list or DataFrame): Attributes for edges.
        y_values (array): Target labels for each point.
    """
    # Convert points to tensor and ensure type is float
    points_tensor = torch.tensor(points, dtype=torch.float)

    # Prepare features tensor by concatenating feature arrays stored in point_data dictionary
    feature_tensors = [torch.tensor(point_data[key], dtype=torch.float).unsqueeze(1) for key in point_data.keys()]
    point_features = torch.cat([points_tensor] + feature_tensors, dim=1)

    # Prepare targets using and next_phi
    y = torch.tensor(y_values, dtype=torch.float)  # Assuming y_values are directly passable to tensor creation

    # Convert edge attributes to tensor
    if isinstance(edge_attributes, list):
        edge_attributes = pd.DataFrame(edge_attributes)
    edge_index = torch.tensor(edge_attributes[['node1', 'node2']].to_numpy().T, dtype=torch.long)

    # Calculate pseudo-coordinates for edge attributes if needed
    pseudo_coords = calculate_pseudo_coordinates(points_tensor, edge_index)
    
    # Construct graph data object
    data = Data(x=point_features, edge_index=edge_index, edge_attr=pseudo_coords, y=y)
    return data

def create_graphs_from_datasets(vtk_pairs):
    """
    Processes pairs of VTK data, where each pair's first item is current data and the second item is next data.

    Args:
        vtk_pairs (list of tuples): List where each tuple contains two sets of data (current and next).

    Returns:
        list: A list of graph data objects, each representing processed graph data from the pairs.
    """
    graph_data_list = []

    for current_data, next_data in vtk_pairs:
        current_points, current_point_data, _, _ = current_data
        next_points, next_point_data, _, next_edge_attributes = next_data

        # Interpolate current point data to next points
        interpolated_point_data = interpolate_features(current_points, current_point_data, next_points)
        interpolated_point_data['theta'] = next_point_data['theta']
        
        # # Y values are directly the phi values from next points
        # y_values = next_point_data['phi']

        # Assuming interpolated_point_data and next_point_data are defined and contain 'phi'
        y_values = np.round(interpolated_point_data['phi']) - np.round(next_point_data['phi'])
        # Keeping only positive values
        y_values = np.maximum(y_values, 0)
        # Rounding to 1 decimal place
        y_values = np.round(y_values, 1)
        
        # Create the graph data using next points, interpolated data, and attributes
        data = create_graph_data(next_points, interpolated_point_data, next_edge_attributes, y_values)
        graph_data_list.append(data)

        progress_percent = (len(graph_data_list) / len(vtk_pairs)) * 100
        print(f"Processing pair {len(graph_data_list)} of {len(vtk_pairs)} ({progress_percent:.2f}%) completed")

    return graph_data_list

graph_data_list = create_graphs_from_datasets(vtk_pairs)

# # Assuming `graph_data_list` is already created
# torch.save(graph_data_list, './generated_data/graph_data_list.pth')
# # Load the graph data list
# graph_data_list = torch.load('./generated_data/graph_data_list.pth')

Processing pair 1 of 4 (25.00%) completed
Processing pair 2 of 4 (50.00%) completed
Processing pair 3 of 4 (75.00%) completed
Processing pair 4 of 4 (100.00%) completed


In [5]:
random_index = random.randint(0, len(graph_data_list) - 1)
graph_data = graph_data_list[random_index]
points = graph_data.x.numpy()[:, :2]
features = graph_data.x.numpy()[:, 2:]  # All features from index 2 to end
edges = graph_data.edge_index.numpy().T
edge_attr = graph_data.edge_attr.numpy()  # Assuming edge_attr contains vector components for each edge
y = graph_data.y.numpy()  # Assuming y contains target values or another feature for each node

myu.plot_graph_components_with_highlights(points, features, edges, edge_attr, y)

NameError: name 'plt' is not defined

In [ ]:
data = myu.combine_graph_data(graph_data_list)
# data.x = data.x[:,:3]
# Print details about the combined graph data
print("Combined Graph Data:")
print(data)

In [ ]:
if torch.isnan(data.x).any() or torch.isnan(data.y).any():
    print("NaNs found in the dataset")
if torch.isinf(data.x).any() or torch.isinf(data.y).any():
    print("Infs found in the dataset")

In [ ]:
print('Before removing nodes based on phi values')
print(data)
data = myu.remove_nodes(data, threshold=0.01)
print('After removing nodes based on phi values')
print(data)

In [ ]:
print('Before removing NaN and Inf nodes')
print(data)
data = myu.remove_nans_and_infs(data)
print('After removing NaN and Inf nodes')
print(data)

In [ ]:
# Ensure y (labels) is of shape [num_nodes]
data.y = data.y.view(-1)  # Ensure y is 1D [num_nodes]

# Number of nodes in the graph
num_nodes = data.x.size(0)  # Infer the number of nodes from x
num_train = int(num_nodes * 0.8)  # 80% of nodes for training
num_test = num_nodes - num_train  # 20% of nodes for testing

# Randomly permute node indices to create random train, validation, and test splits
perm = torch.randperm(num_nodes)

# Initialize masks for train and test sets
train_mask = torch.zeros(num_nodes, dtype=torch.bool)
test_mask = torch.zeros(num_nodes, dtype=torch.bool)

# Assign masks based on the random permutation
train_mask[perm[:num_train]] = True
test_mask[perm[num_train:]] = True

# Assign masks to the Data object
data.train_mask = train_mask
data.test_mask = test_mask

# Optional: Create a validation mask from the training set
num_val = int(num_train * 0.1)  # 10% of the training set for validation
val_mask = torch.zeros(num_nodes, dtype=torch.bool)
val_mask[perm[:num_val]] = True

# Assign validation mask to the Data object
data.val_mask = val_mask

data.num_classes = 2  # Assuming your problem is binary classification

data.y = data.y.float()
# Print the final structured data object
print(data)

In [ ]:
# data.x[:, 2] = np.round(data.x[:, 2], decimals=2)

# data.x = data.x[:, :3]

# data.y = np.round(data.y)

In [ ]:
# Normalize features
data.x = myu.min_max_normalize_features(data.x)
data.edge_attr = myu.min_max_normalize_features(data.edge_attr)
data.y = myu.min_max_normalize_features(data.y)

# # Add gradient features
# data = add_gradient_features(data, data.edge_index)
# print('Done adding gradient features')

# # Optionally, rotate points for augmentation
# data = rotate_points(data, 45)  # Rotate by 45 degrees
# print('Done rotating points')

# # Oversample minority class to balance the dataset
# data = oversample_minority_class(data)
# print('Done oversampling minority class')

# Print data to verify changes
print(data)

In [ ]:
myu.plot_features_and_target(data)

In [ ]:
# Example check for NaNs in your data
if torch.isnan(data.x).any() or torch.isnan(data.y).any():
    print("NaNs found in the dataset")

In [ ]:
# !mkdir ./generated_data
# # Assuming 'data' is your PyTorch Geometric Data object
# torch.save(data, './generated_data/NDD.pt')
# # Load a single data object
# data = torch.load('./generated_data/NDD_per1000.pt')

In [ ]:
print(data)
print(max(data.x[:,2]))
print(data.x)
print(max(data.y))
print(data.y)

In [ ]:
myu.print_random_y_values(data)

In [ ]:
print(data)

In [ ]:
plt.hist(data.y.numpy(), bins=50)
plt.title('Histogram of Model Output Probabilities')
plt.xlabel('Probability')
plt.ylabel('Frequency')
plt.show()

In [ ]:
# def train():
#     model.train()
#     optimizer.zero_grad()
#     out = model(data)
#     loss = F.mse_loss(out[data.train_mask], data.y[data.train_mask].unsqueeze(1))  # Use MSE loss for continuous output
#     loss.backward()
    
#     torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
#     optimizer.step()

# @torch.no_grad()
# def test():
#     model.eval()
#     out = model(data)  # Output shape: [num_nodes, 1]
#     out = out.squeeze()  # Squeeze to match target shape [num_nodes]

#     train_error = F.mse_loss(out[data.train_mask], data.y[data.train_mask]).item()
#     test_error = F.mse_loss(out[data.test_mask], data.y[data.test_mask]).item()
    
#     return train_error, test_error


def train():
    model.train()
    optimizer.zero_grad()

    # Get the model output
    out = model(data)

    # Ensure the output is between 0 and 1 by applying a sigmoid, if not already included in the model
    out_prob = torch.sigmoid(out[data.train_mask])

    # Ensure the target labels are of the correct shape [num_nodes, 1] and type float
    target = data.y[data.train_mask].unsqueeze(1).float()

    # Use binary cross-entropy loss
    loss = F.binary_cross_entropy(out_prob, target)

    # Perform backpropagation and an optimization step
    loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
    optimizer.step()


@torch.no_grad()
def test():
    model.eval()

    # Get the model output
    out = model(data)

    # Ensure the output is between 0 and 1 by applying a sigmoid
    out_prob = torch.sigmoid(out)

    # Calculate binary cross-entropy loss for train and test sets
    train_loss = F.binary_cross_entropy(out_prob[data.train_mask], data.y[data.train_mask].unsqueeze(1).float()).item()
    test_loss = F.binary_cross_entropy(out_prob[data.test_mask], data.y[data.test_mask].unsqueeze(1).float()).item()

    return train_loss, test_loss

# from torch_geometric.data import Batch

# def train():
#     model.train()
#     optimizer.zero_grad()

#     # Assuming 'devices' and 'data' are properly defined
#     data_list = [data.to(device) for device in devices]  # List of data objects for each GPU
#     out = model(data_list)  # Model should handle parallel execution and gathering

#     # Ensure outputs are gathered to a single device, typically cuda:0
#     out_prob = torch.sigmoid(out)  # This should be on a single device after gather
#     device = out_prob.device  # Device where the output is located

#     # Gather targets to the same device as out_prob
#     targets = torch.cat([d.y[d.train_mask].unsqueeze(1).float().to(device) for d in data_list])

#     # Compute loss
#     loss = F.binary_cross_entropy(out_prob, targets)
#     loss.backward()
#     optimizer.step()


# @torch.no_grad()
# def test():
#     model.eval()

#     # Get the model output
#     out = model(data)

#     # Apply sigmoid activation
#     out_prob = torch.sigmoid(out)

#     # Calculate binary cross-entropy loss for train and test sets
#     train_loss = F.binary_cross_entropy(out_prob[data.train_mask], data.y[data.train_mask].unsqueeze(1).float()).item()
#     test_loss = F.binary_cross_entropy(out_prob[data.test_mask], data.y[data.test_mask].unsqueeze(1).float()).item()

#     return train_loss, test_loss

In [ ]:
# from torch_geometric.nn import GCNConv

# class ResidualGCNBlock(torch.nn.Module):
#     def __init__(self, in_channels, out_channels):
#         super().__init__()
#         self.conv1 = GCNConv(in_channels, out_channels)
#         self.conv2 = GCNConv(out_channels, out_channels)
#         if in_channels != out_channels:
#             self.projection = torch.nn.Linear(in_channels, out_channels)
#         else:
#             self.projection = None

#     def forward(self, x, edge_index):
#         identity = x
#         out = F.relu(self.conv1(x, edge_index))
#         out = self.conv2(out, edge_index)
        
#         if self.projection is not None:
#             identity = self.projection(identity)
        
#         out += identity  # Residual connection
#         out = F.relu(out)
#         return out

# class ResidualGCN(torch.nn.Module):
#     def __init__(self, in_channels, hidden_channels, out_channels, num_layers=3, dropout=0.5):
#         super().__init__()
#         self.initial_conv = GCNConv(in_channels, hidden_channels)
#         self.res_blocks = torch.nn.ModuleList([ResidualGCNBlock(hidden_channels, hidden_channels) for _ in range(num_layers - 1)])
#         self.final_conv = GCNConv(hidden_channels, out_channels)
#         self.dropout = dropout

#     def forward(self, data):
#         x, edge_index = data.x, data.edge_index
#         x = F.relu(self.initial_conv(x, edge_index))
#         x = F.dropout(x, p=self.dropout, training=self.training)
        
#         for block in self.res_blocks:
#             x = block(x, edge_index)
#             x = F.dropout(x, p=self.dropout, training=self.training)
        
#         x = self.final_conv(x, edge_index)
#         return x

# def train(model, data, optimizer):
#     model.train()
#     optimizer.zero_grad()
#     out = model(data)  # Shape should be [number of nodes, 1] for regression
#     out = out[data.train_mask]  # Applying mask
#     targets = data.y[data.train_mask].float()  # Ensure targets are float for MSE loss
#     loss = F.mse_loss(out, targets)  # Use MSE loss for continuous output
#     loss.backward()
#     optimizer.step()
#     return loss.item()

# def accuracy(output, labels, threshold=0.1):
#     """Calculate accuracy within a given threshold."""
#     return (output.sub(labels).abs() < threshold).float().mean().item()

# def evaluate(model, data):
#     model.eval()
#     with torch.no_grad():
#         out = model(data).squeeze()  # Squeeze to remove extra dimension from output

#         train_masked_out = out[data.train_mask]
#         test_masked_out = out[data.test_mask]
#         train_labels = data.y[data.train_mask].float()
#         test_labels = data.y[data.test_mask].float()

#         train_acc = accuracy(train_masked_out, train_labels)
#         test_acc = accuracy(test_masked_out, test_labels)

#         return train_acc, test_acc

# # Setup for model training/testing
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# model = ResidualGCN(data.num_node_features, 64, 1, num_layers=3).to(device)  # Assuming the output is 1 for regression
# data = data.to(device)
# optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# # Lists to store training and test accuracy for each epoch
# train_acc_list = []
# test_acc_list = []

# # Example training loop
# for epoch in range(1, 101):
#     train_loss = train(model, data, optimizer)
#     train_acc, test_acc = evaluate(model, data)
#     train_acc_list.append(train_acc)
#     test_acc_list.append(test_acc)
#     print(f'Epoch {epoch}: Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}')



In [ ]:
print(data)

In [ ]:
def check_tensor(tensor, name="Tensor"):
    if torch.isnan(tensor).any() or torch.isinf(tensor).any():
        print(f"{name} contains NaN or Inf.")

# Example usage in your training loop
check_tensor(data.x, "Input Features")

In [ ]:
# import os
# import torch
# import torch.distributed as dist
# import torch.multiprocessing as mp
# import torch.nn as nn
# import torch.optim as optim
# from torch.nn.parallel import DistributedDataParallel as DDP

# def setup(rank, world_size):
#     os.environ['MASTER_ADDR'] = 'localhost'
#     os.environ['MASTER_PORT'] = '29500'
#     dist.init_process_group("nccl", rank=rank, world_size=world_size)
#     torch.cuda.set_device(rank)

# def cleanup():
#     dist.destroy_process_group()

# def train(rank, world_size, model, data, optimizer):
#     torch.cuda.set_device(rank)
#     model.train()

#     # Assuming data is distributed and gathered correctly
#     data = data.to(rank)
#     optimizer.zero_grad()
#     out = model(data)
#     out_prob = torch.sigmoid(out)
#     targets = data.y[data.train_mask].unsqueeze(1).float()
#     loss = nn.functional.binary_cross_entropy(out_prob, targets)
#     loss.backward()
#     optimizer.step()

# @torch.no_grad()
# def test(rank, model, data):
#     model.eval()
#     data = data.to(rank)
#     out = model(data)
#     out_prob = torch.sigmoid(out)
#     loss = nn.functional.binary_cross_entropy(out_prob, data.y[data.test_mask].unsqueeze(1).float())
#     return loss.item()

# def run(rank, world_size, data):
#     setup(rank, world_size)
#     model = GATNet(data).to(rank)
#     model = DDP(model, device_ids=[rank])
#     optimizer = optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-3)

#     for epoch in range(1, 201):
#         train(rank, world_size, model, data, optimizer)
#         if rank == 0:  # Only print from rank 0
#             loss = test(rank, model, data)
#             print(f'Epoch {epoch}, Loss: {loss}')

#     cleanup()

# world_size = torch.cuda.device_count()
# mp.spawn(run,
#          args=(world_size, data),
#          nprocs=world_size,
#          join=True)


In [ ]:
# check and printout device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

print(data)

# GPU Memory Monitoring Function
def print_gpu_memory(device):
    torch.cuda.synchronize(device)
    allocated = torch.cuda.memory_allocated(device)
    reserved = torch.cuda.memory_reserved(device)
    print(f"Allocated memory: {allocated / (1024**3):.3f} GB")
    print(f"Reserved memory: {reserved / (1024**3):.3f} GB")

from NeuralNetworks import SplineCNN_SuperPixelNet, Net, GCNNet, GATNet, HybridGAT_SplineNet
# Initialize the Net model
# model = SplineCNN_SuperPixelNet(data)
model = Net(data)  # Instantiate the model with the data
# model = GCNNet(data)
# model = GATNet(data)
# model = HybridGAT_SplineNet(data)

# Initialize weights
def weights_init(m):
    if hasattr(m, 'weight') and hasattr(m.weight, 'data'):
        torch.nn.init.xavier_uniform_(m.weight.data)
    if hasattr(m, 'bias') and m.bias is not None:
        torch.nn.init.constant_(m.bias.data, 0.0)

# # Apply weights initialization
# model.apply(weights_init)

# Move to the appropriate device
model = model.to(device)
data = data.to(device)

# Set up the optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-3)

# Lists to store training and test accuracy for each epoch
train_acc_list = []
test_acc_list = []

for epoch in range(1, 201):
    train()
    train_acc, test_acc = test()
    train_acc_list.append(train_acc)
    test_acc_list.append(test_acc)
    print(f'Epoch: {epoch:03d}, Train: {train_acc:.4f}, Test: {test_acc:.4f}')

    if epoch % 20 == 0:
        !nvidia-smi
        print_gpu_memory(device)

In [ ]:
# Plot the convergence of training and test accuracy
plt.figure(figsize=(10, 6))
plt.plot(range(1, len(train_acc_list) + 1), train_acc_list, label='Train Accuracy')
plt.plot(range(1, len(test_acc_list) + 1), test_acc_list, label='Test Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Convergence Plot')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
@torch.no_grad()
def evaluate_model_regression(data, model):
    model.eval()
    out = model(data)
    
    actual = data.y[data.test_mask]
    print(actual.size())
    
    # # Calculate Mean Squared Error
    # mse = F.mse_loss(out[data.test_mask], actual)
    # print(f'Test MSE: {mse.item():.4f}')

    # Ensure output is within the [0, 1] range using sigmoid (if not already applied in the model)
    out_prob = torch.sigmoid(out[data.test_mask]).squeeze()  # Apply sigmoid to ensure the outputs are probabilities.
    # Calculate Binary Cross-Entropy
    bce = F.binary_cross_entropy(out_prob, actual)
    print(f'Test BCE: {bce.item():.4f}')

    # Print sample predictions for checking
    print("\nSample Predictions:")
    num_samples = min(10, out[data.test_mask].size(0))
    for i in range(num_samples):
        print(f'Predicted: {out[data.test_mask][i].item():.4f}, Actual: {actual[i].item():.4f}')

evaluate_model_regression(data, model)

In [ ]:
@torch.no_grad()
def evaluate_model_regression(data, model):
    model.eval()
    out = model(data)
    
    actual = data.y[data.test_mask]
    print(actual.size())
    
    # Ensure output is within the [0, 1] range using sigmoid (if not already applied in the model)
    out_prob = torch.sigmoid(out[data.test_mask]).squeeze()  # Apply sigmoid to ensure the outputs are probabilities.
    
    # Calculate Binary Cross-Entropy
    bce = F.binary_cross_entropy(out_prob, actual)
    print(f'Test BCE: {bce.item():.4f}')

    # Calculate accuracy for the entire test dataset
    # predicted_labels = (out_prob > 0.7).int()  # Apply threshold to convert probabilities to binary labels
    predicted_labels = out_prob  # Apply threshold to convert probabilities to binary labels
    correct_predictions = (predicted_labels == actual).float().sum().item()  # Calculate the number of correct predictions
    total_predictions = actual.size(0)  # Total number of predictions
    accuracy = correct_predictions / total_predictions  # Calculate accuracy
    print(f'Test Accuracy: {accuracy:.4f}')

    # Print sample predictions for checking
    print("\nSample Predictions:")
    num_samples = min(20, actual.size(0))  # Limit number of samples to print
    if num_samples > 0:
        sample_indices = torch.randperm(actual.size(0))[:num_samples]  # Randomly pick indices to print
        for i in sample_indices:
            predicted_label = predicted_labels[i].item()
            out_prob_val = out_prob[i].item()
            actual_val = actual[i].item()
            print(f'Pred label: {predicted_label}, Out prob: {out_prob_val:.4f}, Actual: {actual_val:.0f}')

    plt.hist(out_prob.detach().cpu().numpy(), bins=50)
    plt.title('Histogram of Model Output Probabilities')
    plt.xlabel('Probability')
    plt.ylabel('Frequency')
    plt.show()

evaluate_model_regression(data, model)

In [ ]:
@torch.no_grad()
def evaluate_model(data, model, rotation_angle_degrees=-45, num_samples=100):
    model.eval()
    out = model(data)
    
    # Get all test indices
    test_indices = torch.where(data.test_mask)[0]
    
    # Randomly sample from the test indices if the number is greater than num_samples
    if len(test_indices) > num_samples:
        sampled_indices = test_indices[torch.randperm(len(test_indices))[:num_samples]]
    else:
        sampled_indices = test_indices

    # out_prob = torch.sigmoid(out[sampled_indices].squeeze())  # Apply sigmoid to ensure the outputs are probabilities
    # threshold = 0.7  # Define the threshold
    # pred = (out_prob > threshold).int()  # Convert probabilities to binary labels based on the threshold
    pred = torch.sigmoid(out[sampled_indices].squeeze())
    actual = data.y[sampled_indices]
    input_phi = data.x[sampled_indices, 2]  # Assuming the third column is phi
    error = (pred - actual).abs()  # Calculate error as absolute difference

    # # Calculate accuracy for the sampled cases
    # correct = pred.round().eq(actual.round()).sum().item()
    # total = len(sampled_indices)
    # accuracy = correct / total
    # print(f'Test Accuracy (sampled): {accuracy:.4f} for {num_samples} samples')

    # Reverse the rotation for visualization purposes
    angle = np.radians(rotation_angle_degrees)
    rotation_matrix = np.array([
        [np.cos(angle), np.sin(angle)],
        [-np.sin(angle), np.cos(angle)]
    ])

    coords = data.x[sampled_indices][:, :2].cpu().numpy()  # Ensure the first two columns are coordinates
    new_coords = np.dot(coords, rotation_matrix)

    # Visualization of 2D predictions
    fig, ax = plt.subplots(1, 4, figsize=(28, 7), sharex=True, sharey=True)
    scatter = ax[0].scatter(new_coords[:, 0], new_coords[:, 1], c=input_phi.cpu().numpy(), cmap='viridis', s=50)
    ax[0].set_title('Input Phi')
    ax[0].set_xlabel('X Coordinate')
    ax[0].set_ylabel('Y Coordinate')
    plt.colorbar(scatter, ax=ax[0], label='Input Phi Value')

    scatter = ax[1].scatter(new_coords[:, 0], new_coords[:, 1], c=pred.cpu().numpy(), cmap='viridis', s=50)
    ax[1].set_title('Predicted Phi')
    ax[1].set_xlabel('X Coordinate')
    plt.colorbar(scatter, ax=ax[1], label='Predicted Class')

    scatter = ax[2].scatter(new_coords[:, 0], new_coords[:, 1], c=actual.cpu().numpy(), cmap='viridis', s=50)
    ax[2].set_title('Actual Phi')
    ax[2].set_xlabel('X Coordinate')
    plt.colorbar(scatter, ax=ax[2], label='Actual Class')

    # Error visualization
    scatter = ax[3].scatter(new_coords[:, 0], new_coords[:, 1], c=error.cpu().numpy(), cmap='viridis', s=50)
    ax[3].set_title('Absolute error')
    ax[3].set_xlabel('X Coordinate')
    plt.colorbar(scatter, ax=ax[3], label='Error')

    plt.show()

evaluate_model(data, model, 0, num_samples=1000)

In [ ]:
# # credits:
# # how to use DDP module with DDP sampler: https://gist.github.com/sgraaf/5b0caa3a320f28c27c12b5efeb35aa4c
# # how to setup a basic DDP example from scratch: https://pytorch.org/tutorials/intermediate/dist_tuto.html
# import os
# import torch
# import torch.distributed as dist
# import torch.multiprocessing as mp
# import torch
# import torch.nn as nn
# import torch.nn.functional as F
# import torch.optim as optim
# from torchvision import datasets, transforms
# import random

# from torch.nn.parallel import DistributedDataParallel as DDP

# from torch.utils.data import DataLoader, Dataset
# from torch.utils.data.distributed import DistributedSampler
# import math

# def get_dataset():
#     world_size = dist.get_world_size()
#     train_set = datasets.MNIST('./data', train=True, download=True,
#                              transform=transforms.Compose([
#                                  transforms.ToTensor(),
#                                  transforms.Normalize((0.1307,), (0.3081,))
#                              ]))
#     val_set = datasets.MNIST('./data', train=False, download=True,
#                              transform=transforms.Compose([
#                                  transforms.ToTensor(),
#                                  transforms.Normalize((0.1307,), (0.3081,))
#                              ]))
    
#     train_sampler = DistributedSampler(train_set,num_replicas=world_size)
#     val_sampler = DistributedSampler(val_set,num_replicas=world_size)
#     batch_size = int(128 / float(world_size))
#     print(world_size, batch_size)
#     train_loader = DataLoader(
#         dataset=train_set,
#         sampler=train_sampler,
#         batch_size=batch_size
#     )
#     val_loader = DataLoader(
#         dataset=val_set,
#         sampler=val_sampler,
#         batch_size=batch_size
#     )

#     return train_loader, val_loader, batch_size
# class Net(nn.Module):
#     def __init__(self):
#         super(Net, self).__init__()
#         self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
#         self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
#         self.conv2_drop = nn.Dropout2d()
#         self.fc1 = nn.Linear(320, 50)
#         self.fc2 = nn.Linear(50, 10)
#         self.softmax = nn.LogSoftmax(dim=1)

#     def forward(self, x):
#         x = F.relu(F.max_pool2d(self.conv1(x), 2))
#         x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
#         x = x.view(-1, 320)
#         x = F.relu(self.fc1(x))
#         x = F.dropout(x, training=self.training)
#         x = self.fc2(x)
#         x = self.softmax(x)
#         return x
# def average_gradients(model):
#     size = float(dist.get_world_size())
#     for param in model.parameters():
#         dist.all_reduce(param.grad.data, op=dist.ReduceOp.SUM)
#         param.grad.data /= size
# def reduce_dict(input_dict, average=True):
#     world_size = float(dist.get_world_size())
#     names, values = [], []
#     for k in sorted(input_dict.keys()):
#         names.append(k)
#         values.append(input_dict[k])
#     values = torch.stack(values, dim=0)
#     dist.all_reduce(values, op=dist.ReduceOp.SUM)
#     if average:
#         values /= world_size
#     reduced_dict = {k: v for k, v in zip(names, values)}
#     return reduced_dict
# def train(model,train_loader,optimizer,batch_size):
#     device = torch.device(f"cuda:{dist.get_rank()}")
#     train_num_batches = int(math.ceil(len(train_loader.dataset) / float(batch_size)))
#     model.train()
#     # let all processes sync up before starting with a new epoch of training
#     # dist.barrier()
#     criterion = nn.CrossEntropyLoss().to(device)
#     train_loss = 0.0
#     for data, target in train_loader:
#         data, target = data.to(device), target.to(device)
#         optimizer.zero_grad()
#         output = model(data)
#         loss = criterion(output, target)
#         loss.backward()
#         # average gradient as DDP doesn't do it correctly
#         average_gradients(model)
#         optimizer.step()
#         loss_ = {'loss': torch.tensor(loss.item()).to(device)}
#         train_loss += reduce_dict(loss_)['loss'].item()
#         # cleanup
#         # dist.barrier()
#         # data, target, output = data.cpu(), target.cpu(), output.cpu()
#     train_loss_val = train_loss / train_num_batches
#     return train_loss_val
# def accuracy(output, target, topk=(1,)):
#     """Computes the accuracy over the k top predictions for the specified values of k"""
#     with torch.no_grad():
#         maxk = max(topk)
#         batch_size = target.size(0)

#         _, pred = output.topk(maxk, 1, True, True)
#         pred = pred.t()
#         correct = pred.eq(target.view(1, -1).expand_as(pred))

#         res = []
#         for k in topk:
#             correct_k = correct[:k].view(-1).float().sum(0, keepdim=True)
#             res.append(correct_k.div_(batch_size))
#         return res
# def val(model, val_loader,batch_size):
#     device = torch.device(f"cuda:{dist.get_rank()}")
#     val_num_batches = int(math.ceil(len(val_loader.dataset) / float(batch_size)))
#     model.eval()
#     # let all processes sync up before starting with a new epoch of training
#     # dist.barrier()
#     criterion = nn.CrossEntropyLoss().to(device)
#     val_loss = 0.0
#     with torch.no_grad():
#         for data, target in val_loader:
#             data, target = data.to(device), target.to(device)
#             output = model(data)
#             loss = criterion(output, target)
#             loss_ = {'loss': torch.tensor(loss.item()).to(device)}
#             val_loss += reduce_dict(loss_)['loss'].item()
#     val_loss_val = val_loss / val_num_batches
#     return val_loss_val
# def run(rank, world_size):
#     torch.cuda.set_device(rank)
#     torch.cuda.empty_cache()
#     device = torch.device(f"cuda:{rank}")
#     torch.manual_seed(1234)
#     train_loader, val_loader, batch_size = get_dataset()
#     model = Net().to(device)
#     model = nn.SyncBatchNorm.convert_sync_batchnorm(model) # use if model contains batchnorm.
#     model = DDP(model,device_ids=[rank],output_device=rank)
#     optimizer = optim.SGD(model.parameters(),lr=0.01, momentum=0.5)
#     history =  {
#             "rank": rank,
#             "train_loss_val": [],
#             "train_acc_val": [],
#             "val_loss_val": [],
#             "val_acc_val": []
#         }
#     if rank == 0:
#         history = {
#             "rank": rank,
#             "train_loss_val": [],
#             "train_acc_val": [],
#             "val_loss_val": [],
#             "val_acc_val": []
#         }
#     for epoch in range(10):
#         train_loss_val = train(model,train_loader,optimizer,batch_size)
#         val_loss_val = val(model,val_loader,batch_size)
#         if rank == 0:
#             print(f'Rank {rank} epoch {epoch}: {train_loss_val:.2f}/{val_loss_val:.2f}')
#             history['train_loss_val'].append(train_loss_val)
#             history['val_loss_val'].append(val_loss_val)
#     print(f'Rank {rank} finished training')
#     print(history)
#     cleanup(rank)  

# def cleanup(rank):
#     # dist.cleanup()  
#     dist.destroy_process_group()
#     print(f"Rank {rank} is done.")
# def setup_for_distributed(is_master):
#     """
#     This function disables printing when not in master process
#     """
#     import builtins as __builtin__
#     builtin_print = __builtin__.print

#     def print(*args, **kwargs):
#         force = kwargs.pop('force', False)
#         if is_master or force:
#             builtin_print(*args, **kwargs)

#     __builtin__.print = print
# def init_process(
#         rank, # rank of the process
#         world_size, # number of workers
#         fn, # function to be run
#         # backend='gloo',# good for single node
#         # backend='nccl' # the best for CUDA
#         backend='gloo'
#     ):
#     # information used for rank 0
#     os.environ['MASTER_ADDR'] = '127.0.0.1'
#     os.environ['MASTER_PORT'] = '29500'
#     dist.init_process_group(backend, rank=rank, world_size=world_size)
#     dist.barrier()
#     setup_for_distributed(rank == 0)
#     fn(rank, world_size)


# if __name__ == "__main__":
#     world_size = 2
#     processes = []
#     mp.set_start_method("spawn")
#     for rank in range(world_size):
#         p = mp.Process(target=init_process, args=(rank, world_size, run))
#         p.start()
#         processes.append(p)

#     for p in processes:
#         p.join()